# C-More

In [1]:
import pandas as pd

from collections import Counter

from nltk.tokenize import TweetTokenizer
import string

Our goal is to use the HuggingFace API, https://huggingface.co/docs/api-inference/quicktour, to analyse the sentiment of tweets written in Portuguese (PT). We have two options to achieve this:

* get the sentiment directly from the tweets in PT
* translate the tweets to EN and get the sentiment of the translated tweets

We will start by replacing some of the most common abbreviations in social media texts with full words.

### 1. Process text

In [2]:
df = pd.read_pickle('data_galp.pkl')

In [3]:
df.head()

,id,text,retweets,replies,likes,quotes,created_at
0,1550250030476496900,Galp compra por 140 milhões os 25% da Titan So...,0,0,0,0,2022-07-21 22:43:04+00:00
1,1550246814963712001,A Galp convida portugueses a pensar fora do ca...,0,0,0,0,2022-07-21 22:30:17+00:00
2,1550243011350740992,"Mais um ano que renovo o cartão jovem, mais um...",0,0,0,0,2022-07-21 22:15:10+00:00
3,1550242176407425024,@davidkirzner @LiberalNova @LiberalPT Achas? A...,0,0,0,0,2022-07-21 22:11:51+00:00
4,1550240739170439169,#sicnoticias Não sei! Com a França a construir...,0,0,0,0,2022-07-21 22:06:08+00:00


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   id          125 non-null    int64              
 1   text        125 non-null    object             
 2   retweets    125 non-null    int64              
 3   replies     125 non-null    int64              
 4   likes       125 non-null    int64              
 5   quotes      125 non-null    int64              
 6   created_at  125 non-null    datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), int64(5), object(1)
memory usage: 7.0+ KB


We can tokenize our texts to identify some of the most common abbreviations.

In [5]:
# tokenize text and remove punctuation

def tokens_nopunct(text):
    
    tokens = [token for token in TweetTokenizer(reduce_len=True, strip_handles=True).tokenize(text)]
    return [token for token in tokens if token not in string.punctuation]

In [6]:
df['tokens'] = df['text'].map(tokens_nopunct)

In [7]:
counter = Counter()

for tokens in df['tokens']:
    counter.update(tokens)

In [9]:
# most common tokens with length < 4

[token for token in counter.most_common(50) if len(token[0]) < 4]

[('a', 146),
 ('de', 97),
 ('e', 85),
 ('que', 77),
 ('o', 69),
 ('da', 63),
 ('do', 36),
 ('é', 33),
 ('em', 33),
 ('não', 29),
 ('com', 29),
 ('as', 25),
 ('A', 24),
 ('os', 22),
 ('na', 22),
 ('no', 22),
 ('uma', 19),
 ('um', 19),
 ('...', 18),
 ('dos', 17),
 ('EDP', 15),
 ('por', 14),
 ('O', 13),
 ('nos', 12),
 ('ao', 11),
 ('q', 11),
 ('tem', 11),
 ('se', 11),
 ('foi', 11),
 ('à', 9),
 ('só', 8),
 ('me', 8),
 ('Não', 8),
 ('ser', 8),
 ('140', 7),
 ('25', 7),
 ('“', 7),
 ('”', 7)]

In [10]:
# 'q' is one of the most common abbreviations

print(df[df['text'].str.contains(" q ", case=False)]['text'].values)

['A Galp convida portugueses a pensar fora do carro..e quem disser Q isto é só “uma limpeza de imagem”, a Galp, manda “dar banho ao cão”. A máscara 😷 cai. #De-evolution. AbsurdQ https://t.co/TW2XljNvxI'
 '@davidkirzner @LiberalNova @LiberalPT Achas? A moça terminou o curso há pouco tempo e concorreu á Galp... O q estou a dizer é q na Galp trabalham milhares e milhares de pessoas, não é por trabalhar na Galp q somos maus. Nem sequer são eles q decidem preços e afins 😕'
 '@davidkirzner @LiberalNova @LiberalPT David mas eu tenho uma amiga q não é Rica, nunca ligou a Política, é uma excelente pessoa e profissional e trabalha na Galp. Tipo, a Galp tem milhares de funcionários'
 '@Fragoso_1906 sim, a Galp tem uma cena com o continente q tens desconto na fatura e vai para o cartão!'
 '@RuiPaiva5 Já andam todos em pânico. Até lhes tremem as pernas só de pensar no q aconteceu à GALP. https://t.co/puXUVI70da'
 '@tiagojcgodinho @MestredoUnive19 @PSSantiago88 Tens noção q há "ativos" encostados n 

In [11]:
# dictionary with abbreviations to replace

abrev = {" q ": " que "}

In [12]:
# other abbreviations: 'tb'

print(df[df['text'].str.contains(" tb ", case=False)]['text'].values)

["@psocialista @LiberalPT @ppdpsd \nMAL q é feito aos portugueses.\nÑ basta falar na elevada 'carga' fiscal, tb dos baixos salários face a inflação ~9%\nPQ isto tb conta, qd ESTADO é acionista da Galp, em 7%\nhttps://t.co/sYtJAQQAQ4\n'TUGA' SOFRE...\nhttps://t.co/eD0R4EIrXL via @expresso"
 '@jbizarro Acidentes de viação, segundo as estatísticas, tb foram menos, e a Galp, segundo se soube, teve prejuízos avultados, teve menos lucros.🤦']


In [13]:
abrev[" tb "] = " também "

In [14]:
# other abbreviations: 'qd'

print(df[df['text'].str.contains(" qd ", case=False)]['text'].values)

["@psocialista @LiberalPT @ppdpsd \nMAL q é feito aos portugueses.\nÑ basta falar na elevada 'carga' fiscal, tb dos baixos salários face a inflação ~9%\nPQ isto tb conta, qd ESTADO é acionista da Galp, em 7%\nhttps://t.co/sYtJAQQAQ4\n'TUGA' SOFRE...\nhttps://t.co/eD0R4EIrXL via @expresso"]


In [15]:
# other abbreviations to replace

abrev[" qd "] = " quando "
abrev[" ñ "] = " não "
abrev[" pq "] = " porque "
abrev[" hj "] = " hoje "

In [16]:
abrev

{' q ': ' que ',
 ' tb ': ' também ',
 ' qd ': ' quando ',
 ' ñ ': ' não ',
 ' pq ': ' porque ',
 ' hj ': ' hoje '}

This is not an exhaustive list of abbreviations to replace, but it could be updated according to our needs.

Another way of doing this would be using regular expressions to account for all occurrences (and not only lowercase abbreviations with blank spaces at both ends).

We could also try to use a spell checker.

In [17]:
# replace abbreviations by word

def replace_abrev(text, dic):
    for k, v in dic.items():
        text = text.replace(k, v)
    return text

In [18]:
df['text_full'] = df['text'].apply(replace_abrev, dic=abrev)

In [19]:
# example 1

print(df[df['text'].str.contains("milhares e milhares", case=False)]['text'].values, 
      '\n\n', 
      df[df['text_full'].str.contains("milhares e milhares", case=False)]['text_full'].values)

['@davidkirzner @LiberalNova @LiberalPT Achas? A moça terminou o curso há pouco tempo e concorreu á Galp... O q estou a dizer é q na Galp trabalham milhares e milhares de pessoas, não é por trabalhar na Galp q somos maus. Nem sequer são eles q decidem preços e afins 😕'] 

 ['@davidkirzner @LiberalNova @LiberalPT Achas? A moça terminou o curso há pouco tempo e concorreu á Galp... O que estou a dizer é que na Galp trabalham milhares e milhares de pessoas, não é por trabalhar na Galp que somos maus. Nem sequer são eles que decidem preços e afins 😕']


In [20]:
# example 2

print(df[df['text'].str.contains("estatísticas", case=False)]['text'].values, 
      '\n\n', 
      df[df['text_full'].str.contains("estatísticas", case=False)]['text_full'].values)

['@jbizarro Acidentes de viação, segundo as estatísticas, tb foram menos, e a Galp, segundo se soube, teve prejuízos avultados, teve menos lucros.🤦'] 

 ['@jbizarro Acidentes de viação, segundo as estatísticas, também foram menos, e a Galp, segundo se soube, teve prejuízos avultados, teve menos lucros.🤦']


### 2. Sentiment Analysis with a multilingual model fine-tuned for PT

In [37]:
import requests

Before we begin, we need to take into consideration the usage limits of the API. The free version of HugginFace's API is limited to 30000 characters per month: https://huggingface.co/pricing .

In [25]:
# total number of characters of our tweets

sum([len(text) for text in df['text_full']])

20923

Since our tweets have more than 20000 characters, we are going to use a smaller number of them to test these new approaches (otherwise we would quickly get to the monthly limit).

In [27]:
df_sample = df[0:20].copy()

In [28]:
sum([len(text) for text in df_sample['text_full']])

3666

In [30]:
# HuggingFace token

hf_token = ""

In [98]:
# multilingual model for sentiment analysis:
# https://huggingface.co/cardiffnlp/twitter-xlm-roberta-base-sentiment

model = "cardiffnlp/twitter-xlm-roberta-base-sentiment"

In [99]:
API_URL = "https://api-inference.huggingface.co/models/" + model
headers = {"Authorization": "Bearer %s" % (hf_token)}

In [33]:
def apply_model(data):
    payload = dict(inputs=data, options=dict(wait_for_model=True))
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

In [100]:
# test

apply_model("Isto é um teste!")

[[{'label': 'Neutral', 'score': 0.8557949066162109},
  {'label': 'Negative', 'score': 0.08614175766706467},
  {'label': 'Positive', 'score': 0.058063309639692307}]]

In [83]:
%%time

tweets_sentiment = []

for tweet in df_sample['text_full']:
    
    sentiment_result = apply_model(tweet)[0] # the result is a list inside a list
    tweets_sentiment.append({'sentiment': sentiment_result})

Wall time: 12.1 s


In [84]:
df_sample['sentiment'] = pd.DataFrame(tweets_sentiment)

df_sample.head()

,id,text,retweets,replies,likes,quotes,created_at,tokens,text_full,sentiment
0,1550250030476496900,Galp compra por 140 milhões os 25% da Titan So...,0,0,0,0,2022-07-21 22:43:04+00:00,"[Galp, compra, por, 140, milhões, os, 25, da, ...",Galp compra por 140 milhões os 25% da Titan So...,"[{'label': 'Neutral', 'score': 0.7745079398155..."
1,1550246814963712001,A Galp convida portugueses a pensar fora do ca...,0,0,0,0,2022-07-21 22:30:17+00:00,"[A, Galp, convida, portugueses, a, pensar, for...",A Galp convida portugueses a pensar fora do ca...,"[{'label': 'Negative', 'score': 0.836504757404..."
2,1550243011350740992,"Mais um ano que renovo o cartão jovem, mais um...",0,0,0,0,2022-07-21 22:15:10+00:00,"[Mais, um, ano, que, renovo, o, cartão, jovem,...","Mais um ano que renovo o cartão jovem, mais um...","[{'label': 'Positive', 'score': 0.569248735904..."
3,1550242176407425024,@davidkirzner @LiberalNova @LiberalPT Achas? A...,0,0,0,0,2022-07-21 22:11:51+00:00,"[Achas, A, moça, terminou, o, curso, há, pouco...",@davidkirzner @LiberalNova @LiberalPT Achas? A...,"[{'label': 'Negative', 'score': 0.901339948177..."
4,1550240739170439169,#sicnoticias Não sei! Com a França a construir...,0,0,0,0,2022-07-21 22:06:08+00:00,"[#sicnoticias, Não, sei, Com, a, França, a, co...",#sicnoticias Não sei! Com a França a construir...,"[{'label': 'Neutral', 'score': 0.4744070768356..."


In [85]:
# sentiment score ranges from [0, 1] for each label

# top sentiment
df_sample['top_sentiment'] = df_sample['sentiment'].map(lambda sentiment: max(sentiment, key=lambda x: x['score']))

# top score
df_sample['score'] = df_sample['top_sentiment'].map(lambda x: x['score'])

# top label
df_sample['label'] = df_sample['top_sentiment'].map(lambda x: x['label'])

In [86]:
df_sample.head()

,id,text,retweets,replies,likes,quotes,created_at,tokens,text_full,sentiment,top_sentiment,score,label
0,1550250030476496900,Galp compra por 140 milhões os 25% da Titan So...,0,0,0,0,2022-07-21 22:43:04+00:00,"[Galp, compra, por, 140, milhões, os, 25, da, ...",Galp compra por 140 milhões os 25% da Titan So...,"[{'label': 'Neutral', 'score': 0.7745079398155...","{'label': 'Neutral', 'score': 0.7745079398155212}",0.774508,Neutral
1,1550246814963712001,A Galp convida portugueses a pensar fora do ca...,0,0,0,0,2022-07-21 22:30:17+00:00,"[A, Galp, convida, portugueses, a, pensar, for...",A Galp convida portugueses a pensar fora do ca...,"[{'label': 'Negative', 'score': 0.836504757404...","{'label': 'Negative', 'score': 0.8365047574043...",0.836505,Negative
2,1550243011350740992,"Mais um ano que renovo o cartão jovem, mais um...",0,0,0,0,2022-07-21 22:15:10+00:00,"[Mais, um, ano, que, renovo, o, cartão, jovem,...","Mais um ano que renovo o cartão jovem, mais um...","[{'label': 'Positive', 'score': 0.569248735904...","{'label': 'Positive', 'score': 0.5692487359046...",0.569249,Positive
3,1550242176407425024,@davidkirzner @LiberalNova @LiberalPT Achas? A...,0,0,0,0,2022-07-21 22:11:51+00:00,"[Achas, A, moça, terminou, o, curso, há, pouco...",@davidkirzner @LiberalNova @LiberalPT Achas? A...,"[{'label': 'Negative', 'score': 0.901339948177...","{'label': 'Negative', 'score': 0.9013399481773...",0.901340,Negative
4,1550240739170439169,#sicnoticias Não sei! Com a França a construir...,0,0,0,0,2022-07-21 22:06:08+00:00,"[#sicnoticias, Não, sei, Com, a, França, a, co...",#sicnoticias Não sei! Com a França a construir...,"[{'label': 'Neutral', 'score': 0.4744070768356...","{'label': 'Neutral', 'score': 0.4744070768356323}",0.474407,Neutral


In [88]:
df_sample['label'].value_counts()

Neutral     14
Negative     3
Positive     3
Name: label, dtype: int64

In [92]:
print(df_sample[df_sample['label'] == 'Negative']['text_full'].values)

['A Galp convida portugueses a pensar fora do carro..e quem disser Q isto é só “uma limpeza de imagem”, a Galp, manda “dar banho ao cão”. A máscara 😷 cai. #De-evolution. AbsurdQ https://t.co/TW2XljNvxI'
 '@davidkirzner @LiberalNova @LiberalPT Achas? A moça terminou o curso há pouco tempo e concorreu á Galp... O que estou a dizer é que na Galp trabalham milhares e milhares de pessoas, não é por trabalhar na Galp que somos maus. Nem sequer são eles que decidem preços e afins 😕'
 '#galp Um belo exemplo da areia que nos atiram para os olhos quando dizem que a petrolíferas não ganham ( tanto) dinheiro como deviam, #edp mesma coisa, chulos e miseráveis é o que fulanos são, com a conivência governamental https://t.co/nL08rnxd8p']


In [93]:
print(df_sample[df_sample['label'] == 'Positive']['text_full'].values)

['Mais um ano que renovo o cartão jovem, mais um ano em que a câmara não me da acesso ao codigo de desconto \U0001fae0\U0001fae0 ate hoje a parte boa do CJ é o desconto que dão com o cartão jovem galp fica ao preço da gasolina low cost hehehe'
 '@davidkirzner @LiberalNova @LiberalPT David mas eu tenho uma amiga que não é Rica, nunca ligou a Política, é uma excelente pessoa e profissional e trabalha na Galp. Tipo, a Galp tem milhares de funcionários'
 'Asociádevos ao #GALP e fomentemos xuntos o crecemento económico, a inclusión social, a creación de emprego e o apoio á empregabilidade e á mobilidade laboral nas comunidades pesqueiras e acuícolas. \n\n#galp #mar #pesca #Galicia https://t.co/30c2OZqfZW']


Most of the tweets are classified as neutral.

Of the 3 tweets classified as negative, 2 are clearly negative and 1 would be more neutral.

Of the 3 tweets classified as positive, the 3 can be considered positive, but 1 is in Galician and not in Portuguese! - The language identification we used from Twitter is not completely accurate...

### 3. Sentiment Analysis of translated tweets

#### 3.1. Translate tweets with a multilingual model to EN

In [101]:
# multilingual model for translation to EN:
# https://huggingface.co/Helsinki-NLP/opus-mt-mul-en

model = "Helsinki-NLP/opus-mt-mul-en"

In [102]:
API_URL = "https://api-inference.huggingface.co/models/" + model

In [97]:
# test

apply_model("Isto é um teste!")

[{'translation_text': 'This is a test!'}]

In [107]:
%%time

tweets_translation = []

for tweet in df_sample['text_full']:
    
    translation_result = apply_model(tweet)
    tweets_translation.append({'translation': translation_result[0]['translation_text']})

Wall time: 1min 2s


In [110]:
df_sample['translation'] = pd.DataFrame(tweets_translation)

df_sample.head()

,id,text,retweets,replies,likes,quotes,created_at,tokens,text_full,sentiment,top_sentiment,score,label,translation
0,1550250030476496900,Galp compra por 140 milhões os 25% da Titan So...,0,0,0,0,2022-07-21 22:43:04+00:00,"[Galp, compra, por, 140, milhões, os, 25, da, ...",Galp compra por 140 milhões os 25% da Titan So...,"[{'label': 'Neutral', 'score': 0.7745079398155...","{'label': 'Neutral', 'score': 0.7745079398155212}",0.774508,Neutral,Galp buys 140 million to 25% of Titan Solar th...
1,1550246814963712001,A Galp convida portugueses a pensar fora do ca...,0,0,0,0,2022-07-21 22:30:17+00:00,"[A, Galp, convida, portugueses, a, pensar, for...",A Galp convida portugueses a pensar fora do ca...,"[{'label': 'Negative', 'score': 0.836504757404...","{'label': 'Negative', 'score': 0.8365047574043...",0.836505,Negative,The Galp invites Portuguese to think out of th...
2,1550243011350740992,"Mais um ano que renovo o cartão jovem, mais um...",0,0,0,0,2022-07-21 22:15:10+00:00,"[Mais, um, ano, que, renovo, o, cartão, jovem,...","Mais um ano que renovo o cartão jovem, mais um...","[{'label': 'Positive', 'score': 0.569248735904...","{'label': 'Positive', 'score': 0.5692487359046...",0.569249,Positive,"More one year that renews the young card, more..."
3,1550242176407425024,@davidkirzner @LiberalNova @LiberalPT Achas? A...,0,0,0,0,2022-07-21 22:11:51+00:00,"[Achas, A, moça, terminou, o, curso, há, pouco...",@davidkirzner @LiberalNova @LiberalPT Achas? A...,"[{'label': 'Negative', 'score': 0.901339948177...","{'label': 'Negative', 'score': 0.9013399481773...",0.901340,Negative,@davidkirzner @LiberalNova @LiberalPT Achas? T...
4,1550240739170439169,#sicnoticias Não sei! Com a França a construir...,0,0,0,0,2022-07-21 22:06:08+00:00,"[#sicnoticias, Não, sei, Com, a, França, a, co...",#sicnoticias Não sei! Com a França a construir...,"[{'label': 'Neutral', 'score': 0.4744070768356...","{'label': 'Neutral', 'score': 0.4744070768356323}",0.474407,Neutral,#sicnotices I don't know! With France to build...


Other models for translation to try later:

* https://huggingface.co/unicamp-dl/translation-pt-en-t5

* https://huggingface.co/Narrativa/mbart-large-50-finetuned-opus-pt-en-translation

#### 3.2. Analyse sentiment of translated tweets

In [111]:
# EN model for sentiment analysis:
# https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest

model = "cardiffnlp/twitter-roberta-base-sentiment-latest"

We can also try the multilingual model we used before for sentiment analysis: https://huggingface.co/cardiffnlp/twitter-xlm-roberta-base-sentiment .

In [112]:
API_URL = "https://api-inference.huggingface.co/models/" + model

In [113]:
# test

apply_model("Isto é um teste!")

[[{'label': 'Negative', 'score': 0.03316132724285126},
  {'label': 'Neutral', 'score': 0.7641804814338684},
  {'label': 'Positive', 'score': 0.20265817642211914}]]

In [114]:
%%time

tweets_sentiment = []

for tweet in df_sample['text_full']:
    
    sentiment_result = apply_model(tweet)[0] # the result is a list inside a list
    tweets_sentiment.append({'sentiment': sentiment_result})

Wall time: 12.7 s


In [115]:
df_sample['translation_sent'] = pd.DataFrame(tweets_sentiment)

df_sample.head()

,id,text,retweets,replies,likes,quotes,created_at,tokens,text_full,sentiment,top_sentiment,score,label,translation,translation_sent
0,1550250030476496900,Galp compra por 140 milhões os 25% da Titan So...,0,0,0,0,2022-07-21 22:43:04+00:00,"[Galp, compra, por, 140, milhões, os, 25, da, ...",Galp compra por 140 milhões os 25% da Titan So...,"[{'label': 'Neutral', 'score': 0.7745079398155...","{'label': 'Neutral', 'score': 0.7745079398155212}",0.774508,Neutral,Galp buys 140 million to 25% of Titan Solar th...,"[{'label': 'Negative', 'score': 0.080651327967..."
1,1550246814963712001,A Galp convida portugueses a pensar fora do ca...,0,0,0,0,2022-07-21 22:30:17+00:00,"[A, Galp, convida, portugueses, a, pensar, for...",A Galp convida portugueses a pensar fora do ca...,"[{'label': 'Negative', 'score': 0.836504757404...","{'label': 'Negative', 'score': 0.8365047574043...",0.836505,Negative,The Galp invites Portuguese to think out of th...,"[{'label': 'Negative', 'score': 0.335132241249..."
2,1550243011350740992,"Mais um ano que renovo o cartão jovem, mais um...",0,0,0,0,2022-07-21 22:15:10+00:00,"[Mais, um, ano, que, renovo, o, cartão, jovem,...","Mais um ano que renovo o cartão jovem, mais um...","[{'label': 'Positive', 'score': 0.569248735904...","{'label': 'Positive', 'score': 0.5692487359046...",0.569249,Positive,"More one year that renews the young card, more...","[{'label': 'Negative', 'score': 0.024882558733..."
3,1550242176407425024,@davidkirzner @LiberalNova @LiberalPT Achas? A...,0,0,0,0,2022-07-21 22:11:51+00:00,"[Achas, A, moça, terminou, o, curso, há, pouco...",@davidkirzner @LiberalNova @LiberalPT Achas? A...,"[{'label': 'Negative', 'score': 0.901339948177...","{'label': 'Negative', 'score': 0.9013399481773...",0.901340,Negative,@davidkirzner @LiberalNova @LiberalPT Achas? T...,"[{'label': 'Negative', 'score': 0.117106109857..."
4,1550240739170439169,#sicnoticias Não sei! Com a França a construir...,0,0,0,0,2022-07-21 22:06:08+00:00,"[#sicnoticias, Não, sei, Com, a, França, a, co...",#sicnoticias Não sei! Com a França a construir...,"[{'label': 'Neutral', 'score': 0.4744070768356...","{'label': 'Neutral', 'score': 0.4744070768356323}",0.474407,Neutral,#sicnotices I don't know! With France to build...,"[{'label': 'Negative', 'score': 0.108840383589..."


In [116]:
# top sentiment
df_sample['translation_top_sent'] = df_sample['translation_sent'].map(lambda sentiment: max(sentiment, key=lambda x: x['score']))

# top score
df_sample['translation_score'] = df_sample['translation_top_sent'].map(lambda x: x['score'])

# top label
df_sample['translation_label'] = df_sample['translation_top_sent'].map(lambda x: x['label'])

In [118]:
df_sample.head()

,id,text,retweets,replies,likes,quotes,created_at,tokens,text_full,sentiment,top_sentiment,score,label,translation,translation_sent,translation_top_sent,translation_score,translation_label
0,1550250030476496900,Galp compra por 140 milhões os 25% da Titan So...,0,0,0,0,2022-07-21 22:43:04+00:00,"[Galp, compra, por, 140, milhões, os, 25, da, ...",Galp compra por 140 milhões os 25% da Titan So...,"[{'label': 'Neutral', 'score': 0.7745079398155...","{'label': 'Neutral', 'score': 0.7745079398155212}",0.774508,Neutral,Galp buys 140 million to 25% of Titan Solar th...,"[{'label': 'Negative', 'score': 0.080651327967...","{'label': 'Neutral', 'score': 0.8546541333198547}",0.854654,Neutral
1,1550246814963712001,A Galp convida portugueses a pensar fora do ca...,0,0,0,0,2022-07-21 22:30:17+00:00,"[A, Galp, convida, portugueses, a, pensar, for...",A Galp convida portugueses a pensar fora do ca...,"[{'label': 'Negative', 'score': 0.836504757404...","{'label': 'Negative', 'score': 0.8365047574043...",0.836505,Negative,The Galp invites Portuguese to think out of th...,"[{'label': 'Negative', 'score': 0.335132241249...","{'label': 'Neutral', 'score': 0.6241118311882019}",0.624112,Neutral
2,1550243011350740992,"Mais um ano que renovo o cartão jovem, mais um...",0,0,0,0,2022-07-21 22:15:10+00:00,"[Mais, um, ano, que, renovo, o, cartão, jovem,...","Mais um ano que renovo o cartão jovem, mais um...","[{'label': 'Positive', 'score': 0.569248735904...","{'label': 'Positive', 'score': 0.5692487359046...",0.569249,Positive,"More one year that renews the young card, more...","[{'label': 'Negative', 'score': 0.024882558733...","{'label': 'Neutral', 'score': 0.8553747534751892}",0.855375,Neutral
3,1550242176407425024,@davidkirzner @LiberalNova @LiberalPT Achas? A...,0,0,0,0,2022-07-21 22:11:51+00:00,"[Achas, A, moça, terminou, o, curso, há, pouco...",@davidkirzner @LiberalNova @LiberalPT Achas? A...,"[{'label': 'Negative', 'score': 0.901339948177...","{'label': 'Negative', 'score': 0.9013399481773...",0.901340,Negative,@davidkirzner @LiberalNova @LiberalPT Achas? T...,"[{'label': 'Negative', 'score': 0.117106109857...","{'label': 'Neutral', 'score': 0.8400129675865173}",0.840013,Neutral
4,1550240739170439169,#sicnoticias Não sei! Com a França a construir...,0,0,0,0,2022-07-21 22:06:08+00:00,"[#sicnoticias, Não, sei, Com, a, França, a, co...",#sicnoticias Não sei! Com a França a construir...,"[{'label': 'Neutral', 'score': 0.4744070768356...","{'label': 'Neutral', 'score': 0.4744070768356323}",0.474407,Neutral,#sicnotices I don't know! With France to build...,"[{'label': 'Negative', 'score': 0.108840383589...","{'label': 'Neutral', 'score': 0.8227448463439941}",0.822745,Neutral


In [119]:
df_sample['translation_label'].value_counts()

Neutral    20
Name: translation_label, dtype: int64

All the tweets are classified as neutral, mostly due to the poor quality of the PT-EN translation (I think...).

We will now try other models for translating the text from PT to EN.

#### 3.3. Translate tweets with an implementation of T5 for PT-EN

Nota: ver https://github.com/google-research/text-to-text-transfer-transformer/

In [120]:
# T5 model for translation to EN:
# https://huggingface.co/unicamp-dl/translation-pt-en-t5

model = "unicamp-dl/translation-pt-en-t5"

In [121]:
API_URL = "https://api-inference.huggingface.co/models/" + model

In [122]:
# test

apply_model("Isto é um teste!")

[{'translation_text': 'This is a test!'}]

In [123]:
%%time

tweets_translation = []

for tweet in df_sample['text_full']:
    
    translation_result = apply_model(tweet)
    tweets_translation.append({'translation': translation_result[0]['translation_text']})

Wall time: 22.9 s


In [124]:
df_sample['translation_t5'] = pd.DataFrame(tweets_translation)

df_sample.head()

,id,text,retweets,replies,likes,quotes,created_at,tokens,text_full,sentiment,top_sentiment,score,label,translation,translation_sent,translation_top_sent,translation_score,translation_label,translation_t5
0,1550250030476496900,Galp compra por 140 milhões os 25% da Titan So...,0,0,0,0,2022-07-21 22:43:04+00:00,"[Galp, compra, por, 140, milhões, os, 25, da, ...",Galp compra por 140 milhões os 25% da Titan So...,"[{'label': 'Neutral', 'score': 0.7745079398155...","{'label': 'Neutral', 'score': 0.7745079398155212}",0.774508,Neutral,Galp buys 140 million to 25% of Titan Solar th...,"[{'label': 'Negative', 'score': 0.080651327967...","{'label': 'Neutral', 'score': 0.8546541333198547}",0.854654,Neutral,Galp buys for 140 million the 25% of Titan Sol...
1,1550246814963712001,A Galp convida portugueses a pensar fora do ca...,0,0,0,0,2022-07-21 22:30:17+00:00,"[A, Galp, convida, portugueses, a, pensar, for...",A Galp convida portugueses a pensar fora do ca...,"[{'label': 'Negative', 'score': 0.836504757404...","{'label': 'Negative', 'score': 0.8365047574043...",0.836505,Negative,The Galp invites Portuguese to think out of th...,"[{'label': 'Negative', 'score': 0.335132241249...","{'label': 'Neutral', 'score': 0.6241118311882019}",0.624112,Neutral,A Galp invites Portuguese to think outside the...
2,1550243011350740992,"Mais um ano que renovo o cartão jovem, mais um...",0,0,0,0,2022-07-21 22:15:10+00:00,"[Mais, um, ano, que, renovo, o, cartão, jovem,...","Mais um ano que renovo o cartão jovem, mais um...","[{'label': 'Positive', 'score': 0.569248735904...","{'label': 'Positive', 'score': 0.5692487359046...",0.569249,Positive,"More one year that renews the young card, more...","[{'label': 'Negative', 'score': 0.024882558733...","{'label': 'Neutral', 'score': 0.8553747534751892}",0.855375,Neutral,"Another year that renew the young card, anothe..."
3,1550242176407425024,@davidkirzner @LiberalNova @LiberalPT Achas? A...,0,0,0,0,2022-07-21 22:11:51+00:00,"[Achas, A, moça, terminou, o, curso, há, pouco...",@davidkirzner @LiberalNova @LiberalPT Achas? A...,"[{'label': 'Negative', 'score': 0.901339948177...","{'label': 'Negative', 'score': 0.9013399481773...",0.901340,Negative,@davidkirzner @LiberalNova @LiberalPT Achas? T...,"[{'label': 'Negative', 'score': 0.117106109857...","{'label': 'Neutral', 'score': 0.8400129675865173}",0.840013,Neutral,What I am saying is that in Galp there are thous
4,1550240739170439169,#sicnoticias Não sei! Com a França a construir...,0,0,0,0,2022-07-21 22:06:08+00:00,"[#sicnoticias, Não, sei, Com, a, França, a, co...",#sicnoticias Não sei! Com a França a construir...,"[{'label': 'Neutral', 'score': 0.4744070768356...","{'label': 'Neutral', 'score': 0.4744070768356323}",0.474407,Neutral,#sicnotices I don't know! With France to build...,"[{'label': 'Negative', 'score': 0.108840383589...","{'label': 'Neutral', 'score': 0.8227448463439941}",0.822745,Neutral,I don't know! With France building the World's...


In [125]:
text_columns = ['text_full', 'translation', 'translation_t5']

df_sample[text_columns]

,text_full,translation,translation_t5
0,Galp compra por 140 milhões os 25% da Titan So...,Galp buys 140 million to 25% of Titan Solar th...,Galp buys for 140 million the 25% of Titan Sol...
1,A Galp convida portugueses a pensar fora do ca...,The Galp invites Portuguese to think out of th...,A Galp invites Portuguese to think outside the...
2,"Mais um ano que renovo o cartão jovem, mais um...","More one year that renews the young card, more...","Another year that renew the young card, anothe..."
3,@davidkirzner @LiberalNova @LiberalPT Achas? A...,@davidkirzner @LiberalNova @LiberalPT Achas? T...,What I am saying is that in Galp there are thous
4,#sicnoticias Não sei! Com a França a construir...,#sicnotices I don't know! With France to build...,I don't know! With France building the World's...
5,"@RuiSilv30012076 @LiberalNova @LiberalPT Pois,...","@RuiSilv30012076 @LiberalNova @LiberalPT Yes, ...","Because, your friend was from CDS and gets a f..."
6,@davidkirzner @LiberalNova @LiberalPT David ma...,@davidkirzner @LiberalNova @LiberalPT David bu...,"So I have a friend who is not rich, never linked"
7,Estamos a elaborar a nova Estratexia de Desenv...,We are developing the new Local Development St...,We are developing the new Participatory Local ...
8,Asociádevos ao #GALP e fomentemos xuntos o cre...,Join #GALP and together we promote economic gr...,#galp #mar #fish #Galicia https://t.co/30
9,"Os campos de óleo ""supergigantes"" possuem + de...","The ""supergigant"" oil fields have + 5 billion ...","The ""super giant"" oil fields have + than 5 bil..."


In [130]:
print(df_sample[text_columns].iloc[3].values)

['@davidkirzner @LiberalNova @LiberalPT Achas? A moça terminou o curso há pouco tempo e concorreu á Galp... O que estou a dizer é que na Galp trabalham milhares e milhares de pessoas, não é por trabalhar na Galp que somos maus. Nem sequer são eles que decidem preços e afins 😕'
 "@davidkirzner @LiberalNova @LiberalPT Achas? The girl finished the course a little while ago and competed at Galp... What I'm saying is that in Galp thousands and thousands of people work, it's not about working in Galp that we're bad. Not even they are who decide prices and business"
 'What I am saying is that in Galp there are thous']


In [131]:
print(df_sample[text_columns].iloc[13].values)

['#galp Um belo exemplo da areia que nos atiram para os olhos quando dizem que a petrolíferas não ganham ( tanto) dinheiro como deviam, #edp mesma coisa, chulos e miseráveis é o que fulanos são, com a conivência governamental https://t.co/nL08rnxd8p'
 "#galp A beautiful example of the arena that attracted us to our eyes when they say that oillifters don't earn (as much) money as they should, #edp the same thing, noise and misery is what crazy are, with the government conviction https://t.co/nL08rnxd8p"
 'A beautiful example of the sand that throw us to']


In [136]:
print(df_sample[text_columns].iloc[4].values)

['#sicnoticias Não sei! Com a França a construir a Nuclear maior do Mundo em produção de energia.....que vai exportar, a curto prazo não sei porque não á incentivos a aquecimento ect...cozinhar tudo a indução, até energia do vento tomar conta do assunto ... Á EDP ou Galp sem luvas!'
 "#sicnotices I don't know! With France to build the world's largest nuclear power production...... which will export, the short term I don't know why not to the incentives for heating etc...coincin all the induction, until wind energy take account of the matter ... to EDP or Galp without holes!"
 "I don't know! With France building the World's larges"]


In [141]:
# lenght of each original and translated tweet

df_sample[text_columns].applymap(lambda x: len(x))

,text_full,translation,translation_t5
0,95,90,64
1,199,195,51
2,222,249,57
3,274,284,48
4,280,294,53
5,117,118,50
6,206,207,48
7,239,204,73
8,250,222,41
9,278,240,50


For some reason, most of the tweets translated with T5 are truncated, but I cannot understand why...